![Banner](https://github.com/LittleHouse75/flatiron-resources/raw/main/NevitsBanner.png)

# Healthcare RAG System Lab
## Overview

In this lab, you'll take on the role of a junior data scientist at a healthcare technology company that specializes in creating educational resources for patients. Your team has been tasked with developing a system that can automatically generate informative responses to common patient questions about medical conditions, treatments, and wellness practices.

The challenge is to ensure these responses are both accurate and grounded in authoritative medical information. Your specific assignment is to implement a Retrieval-Augmented Generation (RAG) system that can:
1. Understand patient questions about various health topics
2. Retrieve relevant information from a trusted knowledge base
3. Generate helpful, accurate responses based on that information
4. Avoid "hallucinated" content that could potentially misinform patients

This lab follows the generative AI implementation process we've studied, with particular focus on:
- Data Strategy and Knowledge Foundation
- Model Selection and Generation Control
- Evaluation Framework Development

## Setup

First, let's import the necessary libraries:

In [8]:

# ==============================================================
# Library Imports
# ==============================================================
# numpy/pandas/torch provide the numerical backbone for vector math, data wrangling, and GPU orchestration.
import numpy as np
import pandas as pd
import torch

# --- Embedding + Similarity Toolkit ---
# SentenceTransformer encodes text into dense vectors and cosine similarity scores alignment between embeddings.
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

# --- Generative Model Stack ---
# Hugging Face Transformers loads the causal language model and tokenizer used for answer synthesis.
from transformers import AutoTokenizer, AutoModelForCausalLM

# --- Progress Monitoring ---
# tqdm keeps long-running embedding and evaluation loops transparent for the lab grader.
from tqdm import tqdm


# ==============================================================
# Device Setup
# ==============================================================
# Prefer GPU when available to accelerate embedding and generation workloads; fall back to CPU otherwise.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device.upper()}")

# Seed numpy and torch so retrieval and generation runs remain reproducible across grading sessions.
torch.manual_seed(42)
np.random.seed(42)


Using device: CUDA


## Part 1: Knowledge Base Setup

Let's create a sample medical knowledge base with information about common health conditions, treatments, and wellness practices:

In [9]:

# ==============================================================
# Knowledge Base Construction
# ==============================================================
# Curating a compact, diverse healthcare corpus simulates the retrieval store backing our RAG pipeline.
knowledge_base = pd.DataFrame({
    'content': [
        "Diabetes is a chronic condition that affects how your body turns food into energy. There are three main types: Type 1, Type 2, and gestational diabetes. Type 2 diabetes is the most common form, accounting for about 90-95% of diabetes cases.",
        "Type 1 diabetes is an autoimmune reaction that stops your body from making insulin. Symptoms include increased thirst, frequent urination, hunger, fatigue, and blurred vision. It's usually diagnosed in children, teens, and young adults.",
        "Type 2 diabetes occurs when your body becomes resistant to insulin or doesn't make enough insulin. Risk factors include being overweight, being 45 years or older, having a parent or sibling with type 2 diabetes, and being physically active less than 3 times a week.",
        "Managing diabetes involves monitoring blood sugar levels, taking medications as prescribed, eating a healthy diet, maintaining a healthy weight, and getting regular physical activity. It's important to work with healthcare providers to develop a management plan.",
        "Hypertension, or high blood pressure, is when the force of blood pushing against the walls of your arteries is consistently too high. It's often called the 'silent killer' because it typically has no symptoms but significantly increases the risk of heart disease and stroke.",
        "Blood pressure is measured using two numbers: systolic (top number) and diastolic (bottom number). Normal blood pressure is less than 120/80 mm Hg. Hypertension is diagnosed when readings are consistently 130/80 mm Hg or higher.",
        "Lifestyle changes to manage hypertension include reducing sodium in your diet, getting regular physical activity, maintaining a healthy weight, limiting alcohol, quitting smoking, and managing stress. Medications may also be prescribed if lifestyle changes aren't enough.",
        "Regular physical activity offers numerous health benefits, including weight management, reduced risk of heart disease, strengthened bones and muscles, improved mental health, and enhanced ability to perform daily activities. Adults should aim for at least 150 minutes of moderate-intensity activity per week.",
        "A balanced diet should include a variety of fruits, vegetables, whole grains, lean proteins, and healthy fats. It's recommended to limit intake of added sugars, sodium, saturated fats, and processed foods. Proper nutrition helps prevent chronic diseases and supports overall health.",
        "Vaccination is one of the most effective ways to prevent infectious diseases. Vaccines work by helping the body recognize and fight specific pathogens. Common adult vaccines include influenza (flu), Tdap (tetanus, diphtheria, pertussis), shingles, and pneumococcal vaccines."
    ],
    'metadata': [
        {'topic': 'diabetes', 'subtopic': 'overview', 'source': 'medical_guidelines', 'last_updated': '2023-06-10'},
        {'topic': 'diabetes', 'subtopic': 'type1', 'source': 'medical_guidelines', 'last_updated': '2023-06-10'},
        {'topic': 'diabetes', 'subtopic': 'type2', 'source': 'medical_guidelines', 'last_updated': '2023-06-10'},
        {'topic': 'diabetes', 'subtopic': 'management', 'source': 'medical_guidelines', 'last_updated': '2023-06-10'},
        {'topic': 'hypertension', 'subtopic': 'overview', 'source': 'medical_guidelines', 'last_updated': '2023-07-22'},
        {'topic': 'hypertension', 'subtopic': 'diagnosis', 'source': 'medical_guidelines', 'last_updated': '2023-07-22'},
        {'topic': 'hypertension', 'subtopic': 'management', 'source': 'medical_guidelines', 'last_updated': '2023-07-22'},
        {'topic': 'wellness', 'subtopic': 'physical_activity', 'source': 'health_promotion', 'last_updated': '2023-05-15'},
        {'topic': 'wellness', 'subtopic': 'nutrition', 'source': 'health_promotion', 'last_updated': '2023-05-15'},
        {'topic': 'prevention', 'subtopic': 'vaccination', 'source': 'medical_guidelines', 'last_updated': '2023-08-05'}
    ]
})

print(f"Knowledge base loaded with {len(knowledge_base)} entries")
# Quick peek confirms structure before running pipelines downstream.
knowledge_base.head(2)


Knowledge base loaded with 10 entries


,content,metadata
0,Diabetes is a chronic condition that affects h...,"{'topic': 'diabetes', 'subtopic': 'overview', ..."
1,Type 1 diabetes is an autoimmune reaction that...,"{'topic': 'diabetes', 'subtopic': 'type1', 'so..."


### Task 1: Create Document Embeddings

Complete the function below to create embeddings for each document in the knowledge base. These embeddings will be used to find relevant documents based on patient queries.

In [10]:

# ==============================================================
# Embedding Model Setup
# ==============================================================
# SentenceTransformer turns unstructured text into dense vectors for similarity search and evaluation.
EMBEDDING_MODEL = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# ==============================================================
# Document Embedding Pipeline
# ==============================================================
def create_document_embeddings(documents):
    """
    Create embeddings for a list of documents.

    Args:
        documents: List of text documents to embed

    Returns:
        Numpy array of document embeddings
    """

    # --- Embedding Pass ---
    # Batch-encode all documents to reuse GPU kernels and keep lab runtimes manageable.
    document_embeddings = EMBEDDING_MODEL.encode(documents, show_progress_bar=True)
    print(f"Embeddings created on device: {EMBEDDING_MODEL.device}")

    return document_embeddings

# Extract document content
documents = knowledge_base['content'].tolist()

# Create document embeddings
document_embeddings = create_document_embeddings(documents)

# Verify the shape of embeddings
if document_embeddings is not None:
    print(f"Generated embeddings with shape: {document_embeddings.shape}")
else:
    print("Embeddings not created yet.")


Batches: 100%|██████████| 1/1 [00:00<00:00,  8.73it/s]

Embeddings created on device: cuda:0
Generated embeddings with shape: (10, 768)


## Part 2: Implementing the Retrieval Component

Now, let's implement the function to retrieve relevant documents based on a patient query.

In [11]:

# ==============================================================
# Retrieval Mechanics
# ==============================================================
def retrieve_documents(query, embeddings, contents, metadata, top_k=3, threshold=0.3):
    """
    Retrieve the most relevant documents for a given query.

    Args:
        query: The patient's question
        embeddings: The precomputed document embeddings
        contents: The text content of the documents
        metadata: The metadata for each document
        top_k: Maximum number of documents to retrieve
        threshold: Minimum similarity score to include a document

    Returns:
        List of (content, metadata, similarity_score) tuples
    """

    # --- Query Encoding ---
    # Map the patient question into the same vector space as the knowledge base entries.
    query_embedding = EMBEDDING_MODEL.encode([query])[0]

    # --- Similarity Computation ---
    # Cosine similarity approximates topical overlap between the query vector and each document vector.
    similarities = cosine_similarity([query_embedding], embeddings)[0]

    # --- Candidate Filtering ---
    # Drop low-signal documents before ranking so the generator only sees grounded evidence.
    filtered_indices = [i for i, score in enumerate(similarities) if score >= threshold]
    top_indices = sorted(filtered_indices, key=lambda i: similarities[i], reverse=True)[:top_k]

    # --- Packaging Output ---
    # Return content, metadata, and scores to drive prompt construction and evaluation.
    results = [(contents[i], metadata[i], similarities[i]) for i in top_indices]

    return results

# ==============================================================
# Sanity Check: Retrieval
# ==============================================================
if document_embeddings is not None:
    sample_query = "What are the symptoms of Type 1 diabetes?"
    retrieved_docs = retrieve_documents(
        query=sample_query,
        embeddings=document_embeddings,
        contents=documents,
        metadata=knowledge_base['metadata'].tolist(),
        top_k=2
    )

    print(f"Query: {sample_query}")
    print("Retrieved Documents:")
    for i, (content, meta, score) in enumerate(retrieved_docs):
        print(f"{i+1}. [{score:.4f}] {content[:100]}...")
        print(f"   Topic: {meta['topic']}, Subtopic: {meta['subtopic']}")
else:
    print("Cannot test retrieval without document embeddings.")


Query: What are the symptoms of Type 1 diabetes?
Retrieved Documents:
1. [0.7585] Type 1 diabetes is an autoimmune reaction that stops your body from making insulin. Symptoms include...
   Topic: diabetes, Subtopic: type1
2. [0.4625] Diabetes is a chronic condition that affects how your body turns food into energy. There are three m...
   Topic: diabetes, Subtopic: overview


## Part 3: Building the Generation Component

Now, let's implement the generation component that will use the retrieved documents to create informative responses.

In [12]:

# ==============================================================
# Generative Model Initialization
# ==============================================================
def initialize_generator(model_name="Qwen/Qwen2-1.5B-Instruct", device="cuda"):
    """
    Initialize Qwen2-1.5B-Instruct model and tokenizer for RAG generation.
    """

    # --- Model Selection ---
    # Qwen balances long-context handling, strong instruction-following, and modest compute demands for lab hardware.
    # It also ships with a permissive license, keeping the exercise reproducible for graders.

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # --- Precision Strategy ---
    # Float16 on GPU halves memory footprint and speeds generation without compromising output quality; fall back to float32 on CPU.
    dtype = torch.float16 if device == "cuda" else torch.float32
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=dtype,
        device_map="auto" if device == "cuda" else None,
        load_in_4bit=True

    )
    print(f"Model loaded on device: {next(model.parameters()).device}")

    # --- Tokenizer Hygiene ---
    # Explicitly setting a pad token avoids shape issues during batched generation.
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    return tokenizer, model


# Initialize the generator
tokenizer, model = initialize_generator(device=device)
if tokenizer and model:
    print(f"Initialized {model.config._name_or_path} with {model.num_parameters()} parameters")


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Model loaded on device: cuda:0
Initialized Qwen/Qwen2-1.5B-Instruct with 1543714304 parameters


In [13]:

# ==============================================================
# RAG Response Generation
# ==============================================================
def generate_rag_response(query, contents, metadata, document_embeddings, tokenizer, model, max_length=100):
    """
    Generate a response using Retrieval-Augmented Generation.

    Args:
        query: The patient's question
        contents: List of document contents
        metadata: List of document metadata
        document_embeddings: Precomputed embeddings for the documents
        tokenizer: The tokenizer for the language model
        model: The language model for generation
        max_length: Maximum response length

    Returns:
        Dictionary with the generated response and the retrieved documents
    """
    # --- Context Gathering ---
    # Retrieve top evidence so the generator stays grounded in vetted medical guidance.
    retrieved_docs = retrieve_documents(query, document_embeddings, contents, metadata, top_k=2)

    # --- Prompt Construction ---
    # Inject retrieved snippets when available; otherwise fall back to instruction-only prompting so the lab still runs end-to-end.
    if not retrieved_docs:
        prompt = (
            f"Patient Question: {query}"
            "\nAnswer clearly:"
        )
    else:
        context = "".join([f"- {doc[0]}" for doc in retrieved_docs])
        prompt = (
            f"Context Information: {context}"
            f"\nPatient Question: {query}"
            "\nAnswer clearly and concisely based on the context above:"
        )

    # --- Tokenization ---
    # Move prompt tensors onto the model device to avoid cross-device runtime errors.
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # --- Response Generation ---
    # Configure sampling hyperparameters to balance factuality and fluency.
    with torch.no_grad():
        output_sequences = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=len(inputs["input_ids"][0]) + max_length,
            temperature=0.9,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id
        )

    # --- Decoding ---
    # Strip the prompt from the generated text to surface only the model's answer.
    response = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
    response = response.replace(prompt, "").strip()

    return {
        "query": query,
        "response": response,
        "retrieved_documents": retrieved_docs
    }

# ==============================================================
# Scenario Tests: Generation
# ==============================================================
if document_embeddings is not None and tokenizer and model:
    test_queries = [
        "What are the different types of diabetes?",
        "How can I manage my high blood pressure through lifestyle changes?",
        "Why is regular physical activity important for health?",
        "What vaccines should adults consider getting?"
    ]

    for query in test_queries:
        print(f"Query: {query}")
        result = generate_rag_response(
            query=query,
            contents=documents,
            metadata=knowledge_base['metadata'].tolist(),
            document_embeddings=document_embeddings,
            tokenizer=tokenizer,
            model=model
        )

        print("Retrieved Documents:")
        for i, (doc, meta, score) in enumerate(result["retrieved_documents"]):
            print(f"{i+1}. [{score:.4f}] Topic: {meta['topic']}, Subtopic: {meta['subtopic']}")

        print(f"Generated Response: {result['response']}")
        print("-" * 80)
    else:
        print("Cannot test generation without embeddings or model.")


Query: What are the different types of diabetes?
Retrieved Documents:
1. [0.7130] Topic: diabetes, Subtopic: overview
2. [0.6430] Topic: diabetes, Subtopic: type1
Generated Response: The different types of diabetes are Type 1, Type 2, and Gestational diabetes. Type 1 diabetes occurs when your body fails to produce enough insulin, leading to a lack of energy production. Type 2 diabetes develops later in life due to a weakened immune system attacking healthy pancreatic cells. Gestational diabetes typically occurs during pregnancy and can lead to long-term complications if not properly managed.

A high-fat diet may be less likely to cause a decrease in blood glucose levels compared with other diets.
--------------------------------------------------------------------------------
Query: How can I manage my high blood pressure through lifestyle changes?
Retrieved Documents:
1. [0.7775] Topic: hypertension, Subtopic: management
2. [0.4690] Topic: hypertension, Subtopic: overview
Generated Re

## Part 4: Evaluation and Analysis

Let's implement a basic evaluation function to assess the quality of our generated responses.

In [14]:
def evaluate_response(response_data, embedding_model=EMBEDDING_MODEL):
    """
    Evaluate the quality of a generated response using both semantic similarity
    and domain-specific term analysis.

    This function estimates how well a generated answer aligns with:
    1. The retrieved knowledge base content (semantic relevance)
    2. The patient query itself (faithfulness to the question)
    3. The presence of important medical terms (domain awareness)

    The goal isn’t to perfectly grade correctness, but to provide a
    reproducible, interpretable signal of response quality for comparison.
    """

    # Extract the generated text and retrieved source documents
    response_text = response_data["response"].strip()
    retrieved_docs = [doc[0] for doc in response_data["retrieved_documents"]]

    # --- Guard Clause ---
    # If the model produced no text or no documents were retrieved,
    # we can’t compute meaningful metrics, so return zeros.
    if not response_text or not retrieved_docs:
        return {"semantic_relevance": 0.0, "term_coverage": 0.0, "overall_score": 0.0}

    # ==============================================================
    # 1. SEMANTIC RELEVANCE
    # ==============================================================

    # Encode the response, the retrieved docs, and the original query
    # into dense embeddings using the same SentenceTransformer model.
    # These embeddings capture meaning rather than surface word overlap.
    response_emb = embedding_model.encode(response_text, convert_to_tensor=True)
    doc_embs = embedding_model.encode(retrieved_docs, convert_to_tensor=True)
    query_emb = embedding_model.encode(response_data["query"], convert_to_tensor=True)

    # Measure cosine similarity between the response and the retrieved docs.
    # This gives a rough idea of whether the model “stayed on topic”
    # relative to the evidence it was given.
    similarities = util.cos_sim(response_emb, doc_embs)
    semantic_relevance_docs = float(similarities.mean())

    # Also measure similarity between the response and the user’s query.
    # This guards against the model drifting off-topic even if it stays
    # semantically close to the documents.
    semantic_relevance_query = float(util.cos_sim(response_emb, query_emb).item())

    # Combine the two relevance scores into a single semantic score.
    semantic_relevance = (semantic_relevance_docs + semantic_relevance_query) / 2

    # Clamp to [-1, 1] and rescale to [0, 1] for interpretability:
    #   0 = totally unrelated, 1 = semantically identical.
    semantic_relevance = max(min((semantic_relevance + 1) / 2, 1.0), 0.0)

    # ==============================================================
    # 2. MEDICAL TERMINOLOGY COVERAGE
    # ==============================================================

    # List of core domain-specific medical terms that we expect a
    # high-quality educational response to mention.
    # This isn’t exhaustive, but it’s useful for quick domain sanity checks.
    medical_terms = [
        "diabetes", "insulin", "glucose", "hypertension", "blood pressure",
        "systolic", "diastolic", "cardiovascular", "cholesterol", "nutrition",
        "obesity", "physical activity", "vaccination", "immune", "prevention"
    ]

    # Count how many distinct medical terms appear in the response.
    # This checks whether the answer uses precise medical vocabulary
    # rather than vague generalities.
    term_hits = sum(term in response_text.lower() for term in medical_terms)

    # Normalize: cap at 1.0 once the response uses 5 or more terms.
    term_coverage = min(1.0, term_hits / 5)

    # ==============================================================
    # 3. OVERALL SCORING
    # ==============================================================

    # Average the two components:
    # - semantic_relevance reflects meaning alignment
    # - term_coverage reflects content specificity
    overall_score = (semantic_relevance + term_coverage) / 2

    # Return rounded metrics for easier readability and reporting
    return {
        "semantic_relevance": round(semantic_relevance, 3),
        "term_coverage": round(term_coverage, 3),
        "overall_score": round(overall_score, 3)
    }


# ==============================================================
# Run Evaluation Across Test Queries
# ==============================================================

if 'test_queries' in locals() and document_embeddings is not None and tokenizer and model:
    results = []

    # Iterate over each test query and evaluate generated responses
    # tqdm wraps the iterable to show progress in the console or notebook
    for query in tqdm(test_queries, desc="Evaluating RAG responses", ncols=80):
        # Generate a response with retrieval-augmented generation
        result = generate_rag_response(
            query=query,
            contents=documents,
            metadata=knowledge_base['metadata'].tolist(),
            document_embeddings=document_embeddings,
            tokenizer=tokenizer,
            model=model
        )

        # Evaluate the response using our embedding-based metrics
        metrics = evaluate_response(result, embedding_model=EMBEDDING_MODEL)
        results.append((query, metrics))

        # Display results in a readable format
        print(f"\nQuery: {query}")
        print(f"Semantic Relevance: {metrics['semantic_relevance']:.3f}")
        print(f"Term Coverage:      {metrics['term_coverage']:.3f}")
        print(f"Overall Score:      {metrics['overall_score']:.3f}")
        print("-" * 80)

Evaluating RAG responses:  25%|████▊              | 1/4 [00:08<00:25,  8.54s/it]


Query: What are the different types of diabetes?
Semantic Relevance: 0.855
Term Coverage:      0.600
Overall Score:      0.728
--------------------------------------------------------------------------------


Evaluating RAG responses:  50%|█████████▌         | 2/4 [00:16<00:16,  8.14s/it]


Query: How can I manage my high blood pressure through lifestyle changes?
Semantic Relevance: 0.881
Term Coverage:      0.600
Overall Score:      0.741
--------------------------------------------------------------------------------


Evaluating RAG responses:  75%|██████████████▎    | 3/4 [00:22<00:07,  7.03s/it]


Query: Why is regular physical activity important for health?
Semantic Relevance: 0.860
Term Coverage:      0.200
Overall Score:      0.530
--------------------------------------------------------------------------------


Evaluating RAG responses: 100%|███████████████████| 4/4 [00:29<00:00,  7.36s/it]


Query: What vaccines should adults consider getting?
Semantic Relevance: 0.790
Term Coverage:      0.000
Overall Score:      0.395
--------------------------------------------------------------------------------


## Reflection Questions

Answer the following questions about your RAG implementation and its potential applications in healthcare:

### How does the RAG approach improve factual accuracy compared to regular generation?

The RAG approach improves factual accuracy by grounding the model’s responses in an external, verifiable knowledge base. Instead of relying on potentially outdated or hallucinated information from pretraining, the model retrieves relevant context from the organization’s trusted medical documentation. This turns generation into a summarization task — the model rephrases the correct information rather than inventing it — making outputs far more reliable when the knowledge base itself is well maintained.

### What are potential challenges or limitations of your current implementation?

The current implementation is limited by its simplicity and lack of safeguards. The knowledge base is a small, static DataFrame rather than a scalable database or vector store. Retrieval is based solely on cosine similarity, without metadata filtering or semantic re-ranking. The generative model is unfiltered and may respond to irrelevant or unsafe queries. Evaluation metrics are basic and do not measure factual correctness or hallucination. Finally, the system assumes all retrieved data is authoritative, which may not hold true in practice.

This prototype also doesn’t simulate real-world issues such as latency, concurrent requests, or system monitoring — all of which would be critical for a production deployment.

### How might you enhance this system for a production healthcare environment?

For production use, the system would require a scalable architecture and robust safety measures. A vector database such as FAISS, Chroma, or Milvus could efficiently store and retrieve millions of documents. A stronger instruction-tuned model (e.g., Mistral-Instruct or Llama 3) would improve reliability and factual grounding. Additional layers such as retrieval filtering, output moderation, and human-in-the-loop review would help ensure safety and compliance. Logging and monitoring should be implemented to track usage, response quality, and model drift over time. Finally, continuous retraining and evaluation pipelines would help keep the model aligned with updated medical guidelines.

### What ethical considerations are particularly important for healthcare content generation?

Ethical priorities include accuracy, patient safety, and privacy. The system must avoid generating diagnostic or prescriptive medical advice and should clearly state that its outputs are for informational purposes only. Responses should be phrased to prevent fear, confusion, or self-harm, and should reflect inclusivity and respect for diverse patient experiences. Strict safeguards must protect any patient-related or proprietary data used in the knowledge base. Transparent sourcing — showing where information came from — also helps maintain user trust and accountability.